In [1]:
!pip install hazm

In [2]:
import numpy as np
import pandas as pd
import os
import glob
import nltk

nltk.download('punkt')
from nltk.tokenize import word_tokenize

from hazm import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
# reading all source files

parent_dir = "/kaggle/input/farsi-text-2cls/data"
subdirectories = [subdir for subdir in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, subdir))]
print(subdirectories)

# a list for storing data inside
total_data = []


for subdir in subdirectories:
    
    # creating subdirs as string
    subdirectory_path = os.path.join(parent_dir, subdir)
    print(subdirectory_path)
    
    # Get a list of all text files within the subdirectory
    txt_files = glob.glob(os.path.join(subdirectory_path, "*.txt"))
    
    # Iterate over each text file and read them
    for txt_file in txt_files:
        
        with open(txt_file, 'r' , encoding='utf8') as file:
            contents = file.read()
            total_data.append(contents)
            
            
print(total_data[:2])

['اقتصادی', 'سیاسی']
/kaggle/input/farsi-text-2cls/data/اقتصادی
/kaggle/input/farsi-text-2cls/data/سیاسی
['طرح تدارم حضور نظامي انگليس در افغانستان پس از 2014\n فرماندهان ارتش انگليس به دنبال اجرايي کردن برنامه استقرار 200 سرباز انگليسي در ولايت هلمند افغانستان پس از خروج نيروهاي ناتو از اين کشور در سال 2014 ميلادي هستند. \n\u200c به نوشته روزنامه تلگراف، اين پيشنهاد اولين اذعان رسمي دولت انگليس در اين باره است که لندن مي\u200cتواند نيروهاي خود را در اين ولايت که در هفت سال گذشته قريب به 450 سرباز در آن کشته شدند، نگه دارد. \nفرماندهان ارشد ارتش انگليس همچنين نگراني خود را درباره اينکه نيروهاي ارتش افغان آمادگي لازم را در رويارويي با طالبان ندارند، ابراز داشته و گفتند که نارضايتي و بي\u200cميلي در نگه داشتن نيروها در هلمند پس از سال 2014 ميلادي تضعيف کننده روحيه نيروهاي افغان و تاثير انگليس در کابل است. \nمنابع ارشد ارتش انگليس همچنين اعلام کردند: اين پيشنهاد مربوط به استقرار 200 سرباز انگليسي در اردوگاه نظاميان "بسشن" در ولايت هلمند براي فعاليت به عنوان مشاوران نظامي در مقر نيروهاي ار

In [23]:
# separating 2 classes in 2 different lists

politics = total_data[:498]
economics = total_data[498:]


print(len(politics))
print(len(economics))


498
498


In [5]:
# preprocessing

def preprocessor(doc):
    
    # normalizing
    normalizer = Normalizer()
    normalized_doc = normalizer.normalize(doc)
    
    # tokenize into words
    tokens = word_tokenize(normalized_doc)
    
    # deleting punctuations & stop words
    with open('/kaggle/input/farsi-text-2cls/persian_stopwords','r', encoding="utf-8") as file:
        persian_stopwords = file.read().split()
        
    my_punct = ['-' , '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '.','/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '»', '«', '“', '”' , '،']

    for token in tokens:
        if token in my_punct or token in persian_stopwords:
            tokens.remove(token)
    
    # pos tagging
    tagger = POSTagger(model='/kaggle/input/farsi-text-2cls/pos_tagger.model')
    tagged_tokens = tagger.tag(tokens)
        
    # finding roots based on pos
    stemmer = Stemmer()
    lemmatizer = Lemmatizer()

    root_tokens = []
    for token in tagged_tokens:
        if token[1] == 'ADJ':
            root = stemmer.stem(token[0])
        else:
            root = lemmatizer.lemmatize(token[0])
                
        root_tokens.append(root) 
    
    # joining tokens together 
    joined_words = ' '.join(root_tokens)
        
    
        
    return joined_words


In [6]:
preprocessed_politics = []    
preprocessed_economics = []

In [24]:
for doc in politics:                            
    preprocessed_politics.append(preprocessor(doc))

In [25]:
for doc in politics:                            
    preprocessed_economics.append(preprocessor(doc))
    

In [ ]:
print(preprocessed_politics[0])
print(preprocessed_economics[0])

In [ ]:
# split_train_test

X_train = preprocessed_politics[:423] + preprocessed_economics[:423]
y_train = ['politics'] * 423 + ['economics'] * 423

X_test = preprocessed_politics[423:] + preprocessed_economics[423:]
y_test = ['politics'] * 75 + ['economics'] * 75


print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

In [14]:
# delete later

X_train = []
X_train.append(preprocessed_politics[0]) 
X_train.append(preprocessed_economics[0])

y_train = ['politics'] + ['economics'] 

X_test = []
X_test.append(preprocessed_politics[1]) 
X_test.append(preprocessed_economics[1])

y_test = ['politics'] + ['economics'] 

In [17]:
print(len(X_train))                      # del later

2


In [18]:
# convert the text data to numerical type (TF_IDF)

vectorizer = TfidfVectorizer(encoding='utf-8', analyzer='word', token_pattern=r'\b\w+\b')
train_tfidf = vectorizer.fit_transform(X_train)
test_tfidf = vectorizer.transform(X_test)

In [19]:
# building the model

clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter=100, random_state=42)
clf.fit(train_tfidf, y_train)


MLPClassifier(hidden_layer_sizes=(50,), max_iter=100, random_state=42)

In [20]:
#import scipy

# predicting the test set
#test_tfidf = tf.sparse.reorder(test_tfidf)
#test_tfidf = scipy.sparse.csr_matrix.sort_indices(test_tfidf)
y_pred = clf.predict(test_tfidf)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5


In [21]:
type(train_tfidf)

scipy.sparse._csr.csr_matrix